In [32]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Create model architecture with input to condense the demand and then input for states control inputs etc.

In [25]:
in_demand = tf.keras.Input(shape=(20,))
in_red_demand = tf.keras.layers.Dense(5, activation=tf.nn.tanh)(in_demand)

in_other = tf.keras.Input(shape=(6,))

in_0 = tf.keras.layers.concatenate([in_red_demand, in_other])

# Intermediate layers:
interm_1 = tf.keras.layers.Dense(10, activation=tf.nn.tanh)(in_0)
interm_2 = tf.keras.layers.Dense(10, activation=tf.nn.tanh)(in_0)

# splitting:
interm_red_pressure = interm_2[:,:5]
interm_other = interm_2[:,5:]

out_pressure = tf.keras.layers.Dense(20)(interm_red_pressure)
out_other = tf.keras.layers.Dense(5)(interm_other)


In [26]:
model = tf.keras.Model(inputs=[in_demand, in_other], outputs=[out_pressure,out_other])

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 5)            105         input_31[0][0]                   
__________________________________________________________________________________________________
input_32 (InputLayer)           [(None, 6)]          0                                            
__________________________________________________________________________________________________
concatenate_9 (Concatenate)     (None, 11)           0           dense_22[0][0]                   
                                                                 input_32[0][0]               

In [29]:
optim = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=optim, loss='mse')



Create some training data for testing.

In [37]:
n_data = 2000

demand_in = np.random.rand(n_data, 20)
other_in = np.random.rand(n_data, 6)

pressure_out = np.random.rand(n_data, 20)
other_out = np.random.rand(n_data, 5)

In [41]:
history = model.fit([demand_in, other_in], 
                    [pressure_out, other_out], 
                    batch_size=100, 
                    epochs=5)

Train on 2000 samples
Epoch 1/5
2000/2000 [==============================] - 0s 20us/sample - loss: 0.1651 - dense_25_loss: 0.0832 - dense_26_loss: 0.0819
Epoch 2/5
2000/2000 [==============================] - 0s 19us/sample - loss: 0.1651 - dense_25_loss: 0.0832 - dense_26_loss: 0.0819
Epoch 3/5
2000/2000 [==============================] - 0s 20us/sample - loss: 0.1651 - dense_25_loss: 0.0832 - dense_26_loss: 0.0819
Epoch 4/5
2000/2000 [==============================] - 0s 19us/sample - loss: 0.1651 - dense_25_loss: 0.0832 - dense_26_loss: 0.0819
Epoch 5/5
2000/2000 [==============================] - 0s 20us/sample - loss: 0.1651 - dense_25_loss: 0.0832 - dense_26_loss: 0.0819
